In [ ]:
# Run in desktop

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import pandas as pd
import math
import sys
import cv2
import tqdm
from scipy import signal
from jupyterthemes import jtplot

jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
%matplotlib inline

In [ ]:
# Dictionary that maps body keypoints to their respective indices 
# refer to -> https://github.com/Fang-Haoshu/Halpe-FullBody
keypoints_code = {"Nose" : 0, "LShoulder" : 5,  "RShoulder" : 6, "LElbow" : 7, "RElbow" : 8, "LWrist" : 9, 
                  "RWrist" : 10,"LHip" : 11, "RHip" : 12, "LKnee" : 13, "RKnee" : 14,"LAnkle" : 15,
                  "RAnkle" : 16, "Head" : 17, "Neck" : 18, "Hip" : 19, "LBigToe" : 20, "RBigToe" : 21, "LSmallToe" : 22,
                  "RSmallToe" : 23, "LHeel" : 24, "RHeel" : 25}

# Convert dict key to list
keypoints_list = list(keypoints_code.keys())

# Define the path to the dataset
DATASET_PATH = r'C:\Users\User\Desktop\FYP2\Alphapose_results'

# Define global variables
cutoff = 1
fs = 30

# Reading data

In [ ]:
# Synchronize the frames of both videos
def synchronize_data(front_data, front_image_id, side_data, side_image_id):
    # diffs = image_id that present in either videos, but not in both
    diffs = set(front_image_id) ^ (set(side_image_id))
    # Return the data if not in diffs
    return [j for i,j in zip(front_image_id, front_data) if i not in diffs], [j for i,j in zip(side_image_id, side_data) if i not in diffs]

def extract_data(folder_path):
    keypoint_data, keypoint_score, images_id = [], [], []
    previous_frame = '' # Variable that stores the image ID of the previous frame
    previous_score = 0 # Variable that stores the score of the previous frame
    f = open(os.path.join(folder_path, 'alphapose-results.json'))
    all_data = json.load(f)
    for data in all_data:
        # Extract the image ID and score for the current frame
        current_frame = data['image_id']
        current_score = data['score']
        
        # Check if the current frame is the same as the previous frame
        if current_frame == previous_frame:
            # If the current score is higher than the previous score,
            # remove the last data
            if current_score > previous_score:
                keypoint_data.pop()
                keypoint_score.pop()
            else:
                # If the current score is not higher, skip processing
                continue
        else:
            # Store image ID only when current frame is different from the previous frame
            images_id.append(current_frame.split('.')[0])
        keypoints = data['keypoints']
        k = []
        s = []
        # Read only the keypoints we want
        for keypoint_code in keypoints_code.values():
            k.append([keypoints[keypoint_code*3], keypoints[keypoint_code*3+1]])
            s.append(keypoints[keypoint_code*3+2])
        keypoint_score.append(s)
        keypoint_data.append(k)
        # Set current frame as previous frame
        previous_frame = current_frame
        previous_score = current_score
    f.close()
    return keypoint_data, keypoint_score, images_id


def read_data(CLASS_NAME):
    all_data_front, all_data_side, all_keypoint_front_score, all_keypoint_side_score, video_path  = [], [], [], [], []
    print('Reading data of class %s...' % CLASS_NAME)
    dirs = os.listdir(os.path.join(DATASET_PATH, CLASS_NAME))
    # For progress bar
    files_count = int(len(dirs)/2)
    
    # tqdm progress bar
    with tqdm.tqdm(total=files_count, ascii=' >=', file=sys.stdout) as pbar:
        
        for i in range(files_count):
            folder_path_front = os.path.join(DATASET_PATH, CLASS_NAME, '%s_front_%d.mp4' % (CLASS_NAME, i))
            folder_path_side = os.path.join(DATASET_PATH, CLASS_NAME, '%s_side_%d.mp4' % (CLASS_NAME, i))
            video_path.append([os.path.join(folder_path_front, 'AlphaPose_%s_front_%d.mp4' % (CLASS_NAME, i)), 
                               os.path.join(folder_path_side, 'AlphaPose_%s_side_%d.mp4' % (CLASS_NAME, i))])
            
            # Verify that both videos contain the same number of frames
            cap = cv2.VideoCapture(os.path.join(folder_path_front, 'AlphaPose_%s_front_%d.mp4' % (CLASS_NAME, i)))
            length_front = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap = cv2.VideoCapture(os.path.join(folder_path_side, 'AlphaPose_%s_side_%d.mp4' % (CLASS_NAME, i)))
            length_side = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            cap.release()
            assert length_front == length_side, 'Video frames of data %d of class %s are not same!' % (i, CLASS_NAME)

            # Read front video
            front_data, keypoint_front_score, front_image_id  = extract_data(folder_path_front)

            # Read side video
            side_data, keypoint_side_score, side_image_id = extract_data(folder_path_side)

            # Check missing frames
            if len(front_image_id) < length_front or len(side_image_id) < length_side:
                front_data, side_data = synchronize_data(front_data, front_image_id, side_data, side_image_id)
            all_data_front.append(front_data)
            all_data_side.append(side_data)
            all_keypoint_front_score.append(keypoint_front_score)
            all_keypoint_side_score.append(keypoint_side_score)
            pbar.update(1)
            
    return all_data_front, all_data_side, all_keypoint_front_score, all_keypoint_side_score, video_path

In [ ]:
# 1d X_front[0] - data of each subject
# 2d X_front[0][0] - keypoints of each data
# 3d X_front[0][0][0] - x and y coordinates of keypoints

HCG_front, HCG_side, HCG_front_score, HCG_side_score, HCG_video_path = read_data('HCG')
NP_front, NP_side, NP_front_score, NP_side_score, NP_video_path = read_data('NP')
PD_front, PD_side, PD_front_score, PD_side_score, PD_video_path = read_data('PD')

Reading data of class HCG...
100%|==================================================================================| 25/25 [00:03<00:00,  6.78it/s]
Reading data of class NP...
100%|==================================================================================| 11/11 [00:02<00:00,  4.22it/s]
Reading data of class PD...
100%|====================================================================================| 2/2 [00:01<00:00,  1.52it/s]


# Gait features extraction

### Detect walking

In [ ]:
## All extraction functions are writen in batch mode

def get_angle(a, b, c):
    # Calculate angle of three points
    ang = math.degrees(math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

def get_angle_plot(x, y, offset = 1, len_x_axis = 1, len_y_axis = 1):
    # This function just to plot the angle in plt
    angle1 = math.degrees(math.atan2(y[0]-y[1], x[0]-x[1]))
    angle2 = math.degrees(math.atan2(y[2]-y[1], x[2]-x[1]))
    
    theta1 = max(angle1, angle2)
    theta2 = min(angle1, angle2)
    
    angle = theta2 - theta1
    angle = angle + 360 if angle < 0 else angle
    
    return Arc([x[1], y[1]], len_x_axis*offset, len_y_axis*offset, 0, theta1, theta2, label = str(angle)+u"\u00b0")

def detect_walking(all_data, ANGLE_THRESHOLD = 150):
    keyparts = ['RAnkle','RKnee', 'RHip']
    start = True
    walking_frames = []
    for person in all_data:
        for i, data in enumerate(person):
            x = [data[keypoints_list.index(keypart)][0] for keypart in keyparts]
            y = [data[keypoints_list.index(keypart)][1] for keypart in keyparts]
            angle = get_angle([x[0], y[0]], [x[1], y[1]], [x[2], y[2]])
            # Check if angle greater than threshold
            if angle > ANGLE_THRESHOLD and start:
                start = False
                starting_frame = i
            # Check if angle less than threshold and time > 80% of duration
            if angle < ANGLE_THRESHOLD and not start and i > int(len(person) * 0.8):
                ending_frame = i
                start = True
                walking_frames.append([starting_frame, ending_frame])
                break
    return walking_frames

### Detect turning

In [ ]:
def detect_turning(all_data):
    turning_frames = []
    for person in all_data:
        Hip_length = []
        Shoulder_length = []
        for data in person:
            # Calculate the length of hip and shoulder of all frames
            x1 = data[keypoints_list.index('RHip')][0]
            x2 = data[keypoints_list.index('LHip')][0]
            x3 = data[keypoints_list.index('RShoulder')][0]
            x4 = data[keypoints_list.index('LShoulder')][0]
            Hip_length.append(x2-x1)
            Shoulder_length.append(x4-x3)
        # Find the max and min
        max_Hip = Hip_length.index(max(Hip_length))
        min_Hip = Hip_length.index(min(Hip_length))
        max_shoulder = Shoulder_length.index(max(Shoulder_length))
        min_shoulder = Shoulder_length.index(min(Shoulder_length))
        # start of the turning period equal to max length
        # end of the turning period equal to min length
        start = int((max_Hip + max_shoulder)/2)
        end = int((min_Hip + min_shoulder)/2)
        turning_frames.append([start, end])
    return turning_frames

In [ ]:
HCG_walking_frames = detect_walking(HCG_side)
HCG_turning_frames = detect_turning(HCG_front)
NP_walking_frames = detect_walking(NP_side)
NP_turning_frames = detect_turning(NP_front)
PD_walking_frames = detect_walking(PD_side)
PD_turning_frames = detect_turning(PD_front)

### Stride segmentation

In [ ]:
# low pass filter to reduce frequency of points
def butter_lowpass(cutoff, fs, order=5):
    return signal.butter(order, cutoff, fs=fs, btype='lowpass', analog=False)

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = signal.filtfilt(b, a, data)
    return y

In [ ]:
def mean(lst):
    return sum(lst)/len(lst)

def stride_detection(y1, y2, lm_distance, cutoff, fs):
    while True:
        # apply low pass filter
        yf = butter_lowpass_filter(y1, cutoff, fs)
        yb = butter_lowpass_filter(y2, cutoff, fs)
        forward_peaks, _ = signal.find_peaks([p for p in yf], distance = lm_distance)
        backward_peaks, _ = signal.find_peaks([p for p in yb], distance = lm_distance)
        # If there is no any stride detected, try increase cutoff value
        if forward_peaks.shape[0] > 0 and backward_peaks.shape[0] > 0:
            y1 = yf
            y2 = yb
            break
        elif cutoff > 10:
            raise Exception(f'Unable to extract stride') 
        else:
            cutoff += 1
    return forward_peaks, backward_peaks, y1, y2
    
def stride_segmentation(side_data, walking_frames, turning_frames, keypart, lm_distance = 30, display_graph = False):
    stride = []
    stride_ratio = []
    # For display graph purpose
    if display_graph:
        cols = 2
        rows = int(math.ceil((len(side_data))/cols))
        fig = plt.figure(layout='constrained', figsize=[6.4*cols*2, 4.8*rows+4])
        if rows < 2:
            rows += 1
        subfigs = fig.subfigures(rows, cols, hspace=0.07, wspace=0.07)
        fig.suptitle(keypart, fontsize=28)
    for i, person in enumerate(side_data):

        fs = 30 # frequency per second == frame per second
        cutoff = 1
        start_w = walking_frames[i][0]
        start_t = turning_frames[i][0]
        end_w = walking_frames[i][1]
        end_t = turning_frames[i][1]
    
        # Detect stride when moving forward
        x1 = [data[keypoints_list.index(keypart)][0] for data in person[start_w:start_t]]
        y1 = [data[keypoints_list.index(keypart)][1] for data in person[start_w:start_t]]
        
        # Detect stride when moving backward
        x2 = [data[keypoints_list.index(keypart)][0] for data in person[end_t:end_w]]
        y2 = [data[keypoints_list.index(keypart)][1] for data in person[end_t:end_w]]
        
        # Find local min
        
        forward_peaks, backward_peaks, y1, y2 = stride_detection(y1, y2, lm_distance, cutoff, fs)

        # Add one stride to the start and end if not stride detected within the first and last 10% distance of walking path
        if abs(x1[forward_peaks[0]] - x1[0]) > ((max(x1) - min(x1)) * 0.1):
            forward_peaks = np.insert(forward_peaks, 0, 0)
        if abs(x1[forward_peaks[-1]] - x1[-1]) > ((max(x1) - min(x1)) * 0.1):
            forward_peaks = np.append(forward_peaks, len(x1)-1)
        if abs(x2[backward_peaks[0]] - x2[0]) > ((max(x2) - min(x2)) * 0.1):
            backward_peaks = np.insert(backward_peaks, 0, 0)
        if abs(x2[backward_peaks[-1]] - x2[-1]) > ((max(x2) - min(x2)) * 0.1):
            backward_peaks = np.append(backward_peaks, len(x2)-1)
            
        stride.append([forward_peaks.tolist(), backward_peaks.tolist()])
        
        # Calculate the distance in pixels of x-coord of detected strides
        xPeaks_forward = [x1[peak] for peak in forward_peaks]
        xPeaks_backward = [x2[peak] for peak in backward_peaks]
        diffs_forward = [abs(j-i) for i, j in zip(xPeaks_forward[:-1], xPeaks_forward[1:])]
        diffs_backward = [abs(j-i) for i, j in zip(xPeaks_backward[:-1], xPeaks_backward[1:])]
        
        # Calculate the mean of nose to hip distance
        nose_y_forward = [data[keypoints_list.index('Nose')][1] for data in person[start_w:start_t]]
        hip_y_forward = [data[keypoints_list.index('Hip')][1] for data in person[start_w:start_t]]
        nose_to_hip_forward = [abs(y1-y2) for y1,y2 in zip(nose_y_forward,hip_y_forward)]
        
        nose_y_backward = [data[keypoints_list.index('Nose')][1] for data in person[end_t:end_w]]
        hip_y_backward = [data[keypoints_list.index('Hip')][1] for data in person[end_t:end_w]]
        nose_to_hip_backward = [abs(y1-y2) for y1,y2 in zip(nose_y_backward,hip_y_backward)]
        
        # Calculate the ratio between the distance of detected strides and the nose to hip distance
        ratio_forward = mean(diffs_forward)/mean(nose_to_hip_forward)
        ratio_backward = mean(diffs_backward)/mean(nose_to_hip_backward)
        mean_ratio = (ratio_forward + ratio_backward) / 2
        
        stride_ratio.append(mean_ratio)
        
        # For display graph purpose
        if display_graph:
            x1 = list(range(len(y1)))
            x2 = list(range(len(y2)))
            subfigs[int(i/2),i%2].suptitle('Person %d \n Total Stride:%d' % (i, forward_peaks.shape[0]+backward_peaks.shape[0]-2), fontsize = 20)
            axs = subfigs[int(i/2),i%2].subplots(1, 2)
            axs[0].plot(x1, y1)
            xs = [x1[frame] for frame in forward_peaks]
            ys = [y1[frame] for frame in forward_peaks]
            colors = 'red'
            #colors = cm.rainbow(np.linspace(0, 1, len(xs)))
            axs[0].scatter(xs, ys, marker = 'X', color=colors, label='Stride (n): %d' % int(forward_peaks.shape[0]-1))
            axs[0].invert_yaxis()
            axs[0].set_title('Forward')
            axs[0].legend()
            
            axs[1].plot(x2, y2, color='green')
            xs = [x2[frame] for frame in backward_peaks]
            ys = [y2[frame] for frame in backward_peaks]
            #colors = cm.rainbow(np.linspace(0, 1, len(xs)))
            axs[1].scatter(xs, ys, marker = 'X', color=colors, label='Stride (n): %d' % int(backward_peaks.shape[0]-1))
            axs[1].invert_yaxis()
            axs[1].set_title('Backward')
            axs[1].legend()
    if display_graph:
        plt.show()
    return stride, stride_ratio

In [ ]:
HCG_stride_R, HCG_stride_R_ratio = stride_segmentation(HCG_side, HCG_walking_frames, HCG_turning_frames, 'RHeel', display_graph=False)
HCG_stride_L, HCG_stride_L_ratio = stride_segmentation(HCG_side, HCG_walking_frames, HCG_turning_frames, 'LHeel', display_graph=False)
NP_stride_R, NP_stride_R_ratio = stride_segmentation(NP_side, NP_walking_frames, NP_turning_frames, 'RHeel', display_graph=False)
NP_stride_L, NP_stride_L_ratio = stride_segmentation(NP_side, NP_walking_frames, NP_turning_frames, 'LHeel', display_graph=False)
PD_stride_R, PD_stride_R_ratio = stride_segmentation(PD_side, PD_walking_frames, PD_turning_frames, 'RHeel', display_graph=False)
PD_stride_L, PD_stride_L_ratio = stride_segmentation(PD_side, PD_walking_frames, PD_turning_frames, 'LHeel', display_graph=False)

In [ ]:
def step_segmentation(stride_R, stride_L):
    step_numbers = []
    # numbers of strides - 1 equal to steps since two consecutive stride form a step
    for i in range(len(stride_R)):
        step_number = len(stride_R[i][0])-1 + len(stride_L[i][0])-1 + len(stride_R[i][1])-1 + len(stride_L[i][1])-1
        step_numbers.append(step_number)
    return step_numbers

In [ ]:
HCG_step_numbers = step_segmentation(HCG_stride_R, HCG_stride_L)
NP_step_numbers = step_segmentation(NP_stride_R, NP_stride_L)
PD_step_numbers = step_segmentation(PD_stride_R, PD_stride_L)

In [ ]:
def turning_duration_segmentation(turning_frames):
    # number of frames / frame rate = duration
    return [(i[1]-i[0])/fs for i in turning_frames]
HCG_turning_duration = turning_duration_segmentation(HCG_turning_frames)
NP_turning_duration = turning_duration_segmentation(NP_turning_frames)
PD_turning_duration = turning_duration_segmentation(PD_turning_frames)

In [ ]:
def turning_steps_extraction(turning_frames, side_data):
    step_numbers, turning_stride_R, turning_stride_L = [], [], []
    # Same procedure with stride segmentation
    for i, person in enumerate(side_data):
        y1 = [data[keypoints_list.index('RHeel')][1] for data in person[turning_frames[i][0]:turning_frames[i][1]]]
        y2 = [data[keypoints_list.index('LHeel')][1] for data in person[turning_frames[i][0]:turning_frames[i][1]]]
        stride_R, stride_L ,_ ,_ = stride_detection(y1, y2, 30, 5, 30)
        turning_stride_R.append(stride_R)
        turning_stride_L.append(stride_L)
        # Since no add one stride to the start and end so no subtract 1
        step_number = len(stride_R) + len(stride_L)
        step_numbers.append(step_number)
    return step_numbers, turning_stride_R, turning_stride_L
    
HCG_turning_steps, HCG_turning_stride_R, HCG_turning_stride_L = turning_steps_extraction(HCG_turning_frames, HCG_side)
NP_turning_steps, NP_turning_stride_R, NP_turning_stride_L = turning_steps_extraction(NP_turning_frames, NP_side)
PD_turning_steps, PD_turning_stride_R, PD_turning_stride_L = turning_steps_extraction(PD_turning_frames, PD_side)

In [ ]:
def mean_speed_segmentation(walking_frames, turning_frames, stride_R_ratio, stride_L_ratio):
    # Calculate the time of forward + backward
    time = [(((j[0]-i[0])+(i[1]-j[1]))/fs) for i, j in zip(walking_frames, turning_frames)]
    mean_ratio = [(i+j)/2 for i, j in zip(stride_R_ratio, stride_L_ratio)]
    # Speed = mean of the ratio of both feet / total time taken
    mean_speed = [i/j for i, j in zip(mean_ratio, time)]
    return mean_speed
HCG_mean_speed = mean_speed_segmentation(HCG_walking_frames, HCG_turning_frames, HCG_stride_R_ratio, HCG_stride_L_ratio)
NP_mean_speed = mean_speed_segmentation(NP_walking_frames, NP_turning_frames, NP_stride_R_ratio, NP_stride_L_ratio)
PD_mean_speed = mean_speed_segmentation(PD_walking_frames, PD_turning_frames, PD_stride_R_ratio, PD_stride_L_ratio)

In [ ]:
def get_cadence(walking_frames, turning_frames, step_numbers):
    # Convert time to minutes
    time = [(((j[0]-i[0])+(i[1]-j[1]))/fs)/60 for i, j in zip(walking_frames, turning_frames)]
    # steps/total time taken
    cadence = [round(i/j) for i, j in zip(step_numbers, time)]
    return cadence
HCG_cadence = get_cadence(HCG_walking_frames, HCG_turning_frames, HCG_step_numbers)
NP_cadence = get_cadence(NP_walking_frames, NP_turning_frames, NP_step_numbers)
PD_cadence = get_cadence(PD_walking_frames, PD_turning_frames, PD_step_numbers)

In [ ]:
# Create label
label = [1]*len(PD_side) + [0]*(len(HCG_side)+len(NP_side))
# Create dataframe
df = pd.DataFrame(
    {'ratioR': PD_stride_R_ratio + HCG_stride_R_ratio + NP_stride_R_ratio,
     'ratioL': PD_stride_L_ratio + HCG_stride_L_ratio + NP_stride_L_ratio,
     'cadence': PD_cadence + HCG_cadence + NP_cadence,
     'mean_speed': PD_mean_speed + HCG_mean_speed + NP_mean_speed,
     'turning_duration': PD_turning_duration + HCG_turning_duration + NP_turning_duration,
     'turning_steps': PD_turning_steps + HCG_turning_steps + NP_turning_steps,
     'parkinson': label
    })
df

,ratioR,ratioL,cadence,mean_speed,turning_duration,turning_steps,parkinson
0,0.635869,0.583621,68,0.022445,4.533333,7,1
1,0.467937,0.482600,72,0.011388,5.500000,8,1
2,1.309100,1.381996,79,0.117686,1.666667,4,0
3,1.958868,1.853990,80,0.253066,1.200000,2,0
4,1.562694,1.281519,100,0.183104,1.266667,3,0
5,1.734369,1.713451,93,0.267965,1.100000,2,0
6,1.395428,1.343578,86,0.164340,1.366667,3,0
7,1.475825,1.433610,99,0.199276,1.633333,3,0
8,1.985101,1.691550,91,0.252981,1.233333,2,0
9,1.452550,1.399348,90,0.177504,1.566667,2,0


In [ ]:
# Save dataframe to csv file
df.to_csv('dataset.csv', mode='a', index=False, header=False)

### Video visualization

In [ ]:
# Output/Display video with extracted gait features
def video_visualization(video_path, stride_R, stride_L, side_data, walking_frame, turning_frame, turning_R, turning_L, output_path=None, display=True):
    fps = 30
    font = cv2.FONT_HERSHEY_SIMPLEX
    cap = cv2.VideoCapture(video_path[1])
    if output_path:
        fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
        out = cv2.VideoWriter(output_path, fourcc, 30, (1920, 1080))
    if display:
        delay = int((1 / int(fps)) * 1000)
        cv2.namedWindow('full_screen', cv2.WND_PROP_FULLSCREEN)
        cv2.setWindowProperty('full_screen',cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
    fw_r = np.array(stride_R[0]) + walking_frame[0]
    fw_l = np.array(stride_L[0]) + walking_frame[0]
    bw_r = np.array(stride_R[1]) + turning_frame[1]
    bw_l = np.array(stride_L[1]) + turning_frame[1]
    tn_r = turning_R + turning_frame[0]
    tn_l = turning_L + turning_frame[0]
    R_stride = L_stride = turning_duration = count = 0

    while True:
        success, frame = cap.read()
        if success:

            # Draw rectangle
            cv2.rectangle(frame, (0,0), (250,100), (0, 0, 255), -1)
            cv2.rectangle(frame, (250,0), (630,100), (0, 255, 0), -1)
            cv2.rectangle(frame, (630,0), (1010,100), (0, 255, 255), -1)
            cv2.rectangle(frame, (1010,0), (1410,100), (250, 255, 0), -1)
            cv2.rectangle(frame, (1410,0), (1920,100), (145, 215, 255), -1)

            # Write status: sitting/walking/turning
            if count <= walking_frame[0] or count >= walking_frame[1]:
                status = 'sitting'
            elif count >= turning_frame[0] and count <= turning_frame[1]:
                status = 'turning'
            else:
                status = 'walking'

            cv2.putText(frame, status, (10,60), 
                        font, 2, (255,255,255), 5, cv2.LINE_AA)

            # helper function of draw detected strides
            def draw_strides(strides, clear_frame, keypart, colour):
                for stride in strides:
                    if count >= stride and count <= clear_frame + 5:
                        x, y = side_data[stride][keypoints_list.index(keypart)]
                        cv2.ellipse(frame, (int(x), int(y)), (10, 10),
                                    0, 0, 360, colour, -1)

            if count >= turning_frame[0] and count <= turning_frame[1]:
                turning_duration = (count - turning_frame[0])/fps

            draw_strides(fw_r, turning_frame[0], 'RHeel', (0, 255, 0))
            draw_strides(bw_r, walking_frame[1], 'RHeel', (0, 255, 0))
            draw_strides(fw_l, turning_frame[0], 'LHeel', (0, 255, 255)) 
            draw_strides(bw_l, walking_frame[1], 'LHeel', (0, 255, 255))

            draw_strides(tn_r, turning_frame[1], 'RHeel', (255, 0, 0))
            draw_strides(tn_l, turning_frame[1], 'LHeel', (0, 0, 255))

            R_stride = np.sum(fw_r[1:] <= count) + np.sum(bw_r[1:] <= count)
            L_stride = np.sum(fw_l[1:] <= count) + np.sum(bw_l[1:] <= count)
            turning_steps = np.sum(tn_r <= count) + np.sum(tn_l <= count)

            # Write numbers of strides of both leg
            cv2.putText(frame, 'RStride: ' + str(R_stride), (260,60), 
                        font, 2, (0,0,0), 5, cv2.LINE_AA)
            cv2.putText(frame, 'LStride: ' + str(L_stride), (640,60), 
                        font, 2, (0,0,0), 5, cv2.LINE_AA)
            cv2.putText(frame, 'Tn_steps: ' + str(turning_steps), (1020,60), 
                        font, 2, (0,0,0), 5, cv2.LINE_AA)
            cv2.putText(frame, 'Tn_dur: %.2f s' % turning_duration, (1420,60), 
                        font, 2, (0,0,0), 5, cv2.LINE_AA)
            count += 1
            if output_path:
                out.write(frame)
            if display:
                cv2.imshow('full_screen', frame)
                key = cv2.waitKey(delay)
                if key == ord('q'):
                    break
                if key == ord(' '):
                    cv2.waitKey(-1)
        else:
            break
    if display:
        cv2.destroyAllWindows()
    if output_path:
        out.release()
    cap.release()

# Function that output all the videos
def output_all_videos(videos_path, stride_R, stride_L, side_data, walking_frames, turning_frames, turning_R, turning_L, class_name):
    print(f'Outputing videos of {class_name} class')
    with tqdm.tqdm(total=len(videos_path), ascii=' >=', file=sys.stdout) as pbar:
        for i in range(len(side_data)):
            output_path = f'C:\\Users\\User\\Desktop\\FYP2\\ouput\\{class_name}_{i}.mp4'
            video_visualization(videos_path[i], stride_R[i], stride_L[i], side_data[i], walking_frames[i], turning_frames[i], turning_R[i], turning_L[i], output_path=output_path, display=False)
            pbar.update(1)
            

In [ ]:
# Below all are just spare code for convenience
'''person = 1
video_visualization(HCG_video_path[person], HCG_stride_R[person], HCG_stride_L[person], HCG_side[person], HCG_walking_frames[person], 
      HCG_turning_frames[person], HCG_turning_stride_R[person], HCG_turning_stride_L[person])'''

'person = 1\nvideo_visualization(HCG_video_path[person], HCG_stride_R[person], HCG_stride_L[person], HCG_side[person], HCG_walking_frames[person], \n      HCG_turning_frames[person], HCG_turning_stride_R[person], HCG_turning_stride_L[person])'

In [ ]:
'''person = 0
video_visualization(NP_video_path[person], NP_stride_R[person], NP_stride_L[person], NP_side[person], NP_walking_frames[person], 
      NP_turning_frames[person], NP_turning_stride_R[person], NP_turning_stride_L[person])'''

'person = 0\nvideo_visualization(NP_video_path[person], NP_stride_R[person], NP_stride_L[person], NP_side[person], NP_walking_frames[person], \n      NP_turning_frames[person], NP_turning_stride_R[person], NP_turning_stride_L[person])'

In [ ]:
'''person = 0
video_visualization(PD_video_path[person], PD_stride_R[person], PD_stride_L[person], PD_side[person], PD_walking_frames[person], 
      PD_turning_frames[person], PD_turning_stride_R[person], PD_turning_stride_L[person])'''

'person = 0\nvideo_visualization(PD_video_path[person], PD_stride_R[person], PD_stride_L[person], PD_side[person], PD_walking_frames[person], \n      PD_turning_frames[person], PD_turning_stride_R[person], PD_turning_stride_L[person])'

In [ ]:
'''output_all_videos(HCG_video_path, HCG_stride_R, HCG_stride_L, HCG_side, HCG_walking_frames, 
      HCG_turning_frames, HCG_turning_stride_R, HCG_turning_stride_L, 'HCG')
output_all_videos(PD_video_path, PD_stride_R, PD_stride_L, PD_side, PD_walking_frames, 
      PD_turning_frames, PD_turning_stride_R, PD_turning_stride_L, 'PD')
output_all_videos(NP_video_path, NP_stride_R, NP_stride_L, NP_side, NP_walking_frames, 
      NP_turning_frames, NP_turning_stride_R, NP_turning_stride_L, 'NP')'''

"output_all_videos(HCG_video_path, HCG_stride_R, HCG_stride_L, HCG_side, HCG_walking_frames, \n      HCG_turning_frames, HCG_turning_stride_R, HCG_turning_stride_L, 'HCG')\noutput_all_videos(PD_video_path, PD_stride_R, PD_stride_L, PD_side, PD_walking_frames, \n      PD_turning_frames, PD_turning_stride_R, PD_turning_stride_L, 'PD')\noutput_all_videos(NP_video_path, NP_stride_R, NP_stride_L, NP_side, NP_walking_frames, \n      NP_turning_frames, NP_turning_stride_R, NP_turning_stride_L, 'NP')"